In [139]:
import pandas as pd
import numpy as np
from headers import *
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
import statsmodels.api as sm

In [140]:
df = pd.read_csv('../data/combined_inner.csv')
X = df[ratioKeys + relativeRatioKeys]
X = sm.add_constant(X)

In [141]:
models = {}
importantDict = {}

In [142]:
for yLabel in yAlpha:
    y = df[yLabel]
    results = sm.OLS(y, X).fit()
    print(results.summary())
    models[yLabel] = results

                            OLS Regression Results                            
Dep. Variable:             alpha1Year   R-squared:                       0.011
Model:                            OLS   Adj. R-squared:                  0.010
Method:                 Least Squares   F-statistic:                     18.33
Date:                Thu, 22 May 2025   Prob (F-statistic):           7.55e-59
Time:                        20:51:24   Log-Likelihood:                -10100.
No. Observations:               30261   AIC:                         2.024e+04
Df Residuals:                   30242   BIC:                         2.040e+04
Df Model:                          18                                         
Covariance Type:            nonrobust                                         
                                   coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------
const           

In [143]:
for yLabel in yAlpha:
    model = models[yLabel]
    pvalues = model.pvalues
    important = pvalues.where(lambda x: x < 0.05).dropna()
    print(f'For {yLabel} significant (p<=0.05) are:')
    print(important)
    print('-----------------')
    for key in important.index:
        if key not in importantDict:
            importantDict[key] = 0
        importantDict[key] += 1

For alpha1Year significant (p<=0.05) are:
const                          1.198909e-40
psRatio                        3.688051e-04
evToEbit                       2.358552e-11
evToSales                      5.347502e-07
evToGrossProfit                5.987812e-14
priceToGrossProfit             1.209579e-06
psRatio_relative               1.580229e-07
peRatio_relative               5.131742e-07
evToEbitda_relative            1.377222e-03
evToEbit_relative              6.545145e-06
evToSales_relative             2.357406e-03
priceToGrossProfit_relative    2.328341e-02
dtype: float64
-----------------
For adjustedAlpha1Year significant (p<=0.05) are:
const                          2.609905e-51
evToEbit                       3.214245e-11
evToSales                      4.699292e-03
evToGrossProfit                4.532064e-12
priceToGrossProfit             1.566976e-05
psRatio_relative               3.700454e-07
peRatio_relative               1.095668e-05
priceToBook_relative           4.703018

How many times each column was classified as significant

In [144]:
importantDict = dict(sorted(importantDict.items(), key=lambda item: item[1], reverse=True))
print(importantDict)

{'const': 8, 'evToEbit': 8, 'evToSales': 8, 'evToGrossProfit': 8, 'priceToGrossProfit': 8, 'psRatio_relative': 8, 'evToEbit_relative': 8, 'peRatio_relative': 7, 'priceToGrossProfit_relative': 7, 'psRatio': 6, 'evToSales_relative': 6, 'evToEbitda_relative': 4, 'priceToBook': 4, 'priceToFreeCashFlow': 2, 'priceToBook_relative': 1, 'evToGrossProfit_relative': 1, 'priceToFreeCashFlow_relative': 1}


Linear predictions

In [145]:
for yLabel in yAlpha:
    y = df[yLabel]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2)
    results = sm.OLS(y_train, X_train).fit()
    model = results.model
    res = results.get_prediction(X_test).predicted
    print(f'R2 score for {yLabel} is {r2_score(y_test, res)}')

R2 score for alpha1Year is 0.01080765005422235
R2 score for adjustedAlpha1Year is 0.010334042681817035
R2 score for equalAlpha1Year is 0.011767790548936508
R2 score for equalAdjustedAlpha1Year is 0.011395617752555376
R2 score for alpha2Year is 0.007062470943189458
R2 score for adjustedAlpha2Year is 0.006408725938267246
R2 score for equalAlpha2Year is 0.008974033865092435
R2 score for equalAdjustedAlpha2Year is 0.008367229926764086


R2 is basically 0 everywhere that means
## Linear models are not reliable regressor in this case
Will use SVM as classificator 
** See svm_predict.py, use_svm_models.py **